In [1]:
import pandas as pd
import os
import sys

module_path = os.path.abspath(os.path.join('../../py-conjugated/'))
if module_path not in sys.path:
    sys.path.append(module_path)
import network_utils as nuts

In [2]:
#Take in data as a dataframe for easy preprocessing
total_df = pd.read_excel('/Users/wesleytatum/Desktop/py-conjugated/data/OFET_total_df.xlsx')
print (total_df.shape)
total_df.head()

(37, 43)


,Unnamed: 0,Anneal_time,Anneal_temp,Substrate,Device,p1_area_avg,p1_area_stdev,p1_filled_area_avg,p1_filled_area_stdev,p1_extent_avg,...,p2_Orient_avg,p2_Orient_stdev,p2_Perim_avg,p2_Perim_stdev,p1_fraction,p2_fraction,mu_lin_avg,r_lin_avg,on_off_avg,Vt_avg
0,0,10,100,3,1,15.060429,13.825293,15.094867,13.930399,0.760417,...,0.273477,1.129281,18.169495,10.105495,0.496546,0.503454,0.027205,0.751935,83456.5773,-18.385927
1,1,10,100,3,3,24.529301,21.770880,24.550095,21.813084,0.711369,...,0.153039,1.105821,24.341023,13.173599,0.470745,0.529255,0.038410,0.748463,133461.6840,-19.986533
2,2,10,100,3,5,31.114610,28.907570,31.171285,29.100225,0.725212,...,0.115643,1.031045,24.335218,14.999830,0.503413,0.496587,0.025188,0.747956,132094.0970,-20.648421
3,3,10,100,3,5,16.867677,12.876565,16.895245,12.918527,0.753319,...,0.101798,0.957505,14.855980,8.463103,0.501345,0.498655,0.025188,0.747956,132094.0970,-20.648421
4,4,10,100,4,3,23.082537,18.354319,23.096438,18.379911,0.720716,...,0.055038,1.075792,15.034620,9.558353,0.439411,0.560589,0.032493,0.732737,245995.6098,-21.035315


In [6]:
Y = total_df[['mu_lin_avg', 'r_lin_avg', 'on_off_avg', 'Vt_avg']] #target features to be predicted
X = total_df.drop(['Unnamed: 0', 'Substrate', 'Device', 'mu_lin_avg', 'r_lin_avg', 'on_off_avg', 'Vt_avg'], axis = 1) #input features used to make prediction

In [8]:
x,y = X.shape

normed_X = pd.DataFrame()

X_norm_key = {}

for colname, coldata in X.iteritems():
    max_val = coldata.max()
    min_val = coldata.min()
    
    X_norm_key[colname] = [min_val, max_val]
    
    normed_col = (coldata - min_val) / (max_val - min_val)
    normed_X[colname] = normed_col
    
print (X_norm_key)
normed_X.head()    

{'Anneal_time': [0, 30], 'Anneal_temp': [0, 175], 'Substrate': [1, 8], 'Device': [1, 8], 'p1_fraction': [0.2485966810230451, 0.5888691307845378], 'p2_fraction': [0.1251534781320124, 0.4930661832222824], 'p3_fraction': [0.2165579794101096, 0.3968688126390562], 'p1_area_avg': [29.50933786078098, 175.6946564885496], 'p1_area_stdev': [46.38652305896206, 1170.579809327209], 'p1_filled_area_avg': [30.57555178268251, 194.4179104477612], 'p1_filled_area_stdev': [49.58336428191448, 1345.46478563035], 'p1_extent_avg': [0.493128951863037, 0.6241187149650081], 'p1_extent_stdev': [0.1404857288988411, 0.2166691369983912], 'p1_MajorAL_avg': [9.177927908753725, 20.35929048563928], 'p1_MajorAL_stdev': [6.854708787015648, 34.31649493070623], 'p1_MinorAL_avg': [4.682076682149734, 9.22944322343303], 'p1_MinorAL_stdev': [3.604720637690147, 13.47803449164003], 'p1_Ecc_avg': [0.794558920796938, 0.911964589525746], 'p1_Ecc_stdev': [0.09735926972520725, 0.1604045368548698], 'p1_Orient_avg': [-0.451050996652246

,Anneal_time,Anneal_temp,Substrate,Device,p1_fraction,p2_fraction,p3_fraction,p1_area_avg,p1_area_stdev,p1_filled_area_avg,...,p3_MajorAL_avg,p3_MajorAL_stdev,p3_MinorAL_avg,p3_MinorAL_stdev,p3_Ecc_avg,p3_Ecc_stdev,p3_Orient_avg,p3_Orient_stdev,p3_Perim_avg,p3_Perim_stdev
0,0.5,0.571429,0.428571,0.142857,0.379893,0.741510,0.042227,0.747318,0.416992,0.747056,...,0.369497,0.224733,0.470751,0.459038,0.116318,0.683351,0.587973,0.725402,0.385802,0.214615
1,0.5,0.571429,0.428571,0.428571,0.299771,0.662438,0.354767,0.338403,0.119074,0.308619,...,0.662014,0.720357,0.679762,0.839553,0.189330,0.745316,0.668887,0.752081,0.756401,0.786992
2,0.5,0.571429,0.428571,0.857143,0.122273,0.555842,0.907236,0.637064,0.178141,0.581681,...,0.683919,0.649905,0.674337,1.000000,0.344826,0.548846,0.461326,0.740511,0.771184,1.000000
3,1.0,0.571429,0.857143,0.142857,0.019406,0.848362,0.504491,0.832873,0.498779,0.868876,...,0.638331,0.686998,0.500720,0.480947,0.318441,0.466093,0.140235,0.431908,0.557095,0.404926
4,1.0,0.571429,0.857143,0.428571,0.338566,0.737923,0.127534,0.833337,0.351669,0.782907,...,0.514102,0.315159,0.493588,0.259797,0.192689,0.571745,0.507254,0.653533,0.434879,0.148635


In [23]:
def df_normalizer(dataframe):
    
    df = dataframe
    
    normed_df = pd.DataFrame()

    df_norm_key = {}

    for colname, coldata in df.iteritems():
        max_val = coldata.max()
        min_val = coldata.min()

        df_norm_key[colname] = [min_val, max_val]

        normed_col = (coldata - min_val) / (max_val - min_val)
        normed_df[colname] = normed_col
        
    return normed_df, df_norm_key 

In [10]:
df, key = nuts.df_MinMax_normalize(Y)

print (key)
df.head()

{'mu_lin_avg': [0.002176017279821623, 0.04063017598343683], 'r_lin_avg': [0.484691754, 0.8632616752434137], 'on_off_avg': [2.212495309913667, 245995.6098], 'Vt_avg': [-31.51137623971937, 52.10723525219907]}


,mu_lin_avg,r_lin_avg,on_off_avg,Vt_avg
0,0.650868,0.705927,0.339254,0.156968
1,0.942271,0.696756,0.542533,0.137826
2,0.598439,0.695419,0.536973,0.129911
3,0.598439,0.695419,0.536973,0.129911
4,0.788403,0.655215,1.000000,0.125284


In [8]:
def df_denormalize(normed_df, norm_key):
    
    denormed_df = pd.DataFrame()
    
    for colname, coldata in normed_df.iteritems():
        mn = norm_key[colname][0]
        mx = norm_key[colname][1]
        
        denormed_col = (coldata * (mx - mn)) + mn
        
        denormed_df[colname] = denormed_col
        
    return denormed_df

In [11]:
test = nuts.df_MinMax_denormalize(df, key)

test.head()

,mu_lin_avg,r_lin_avg,on_off_avg,Vt_avg
0,0.027205,0.751935,83456.5773,-18.385927
1,0.038410,0.748463,133461.6840,-19.986533
2,0.025188,0.747956,132094.0970,-20.648421
3,0.025188,0.747956,132094.0970,-20.648421
4,0.032493,0.732737,245995.6098,-21.035315
